# Factores de riesgo
Este proyecto se corresponde al trabajo final del curso de Data Science dictado por la Fundación YPF en el año 2024.

## Objetivos del proyecto
Desarrollar un modelo predictivo para identificar individuos con alto riesgo de desarrollar enfermedades crónicas (como hipertensión, diabetes, colesterol alto) basándose en sus comportamientos y condiciones de vida.

## Setup de archivos

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as skl

from utils.funcions import (
    calcular_nulos_y_porcentaje
)

### Lectura de datasets

In [ ]:
# Conecto drive
from google.colab import drive
drive.mount('/content/drive')

# Leo el archivo
friesgo_2018 = pd.read_csv('/content/drive/MyDrive/INGENIAS/TrabajoP/bd/friesgo2018.txt', delimiter = '|')

In [2]:
## Leo el archivo localmente
friesgo_2018 = pd.read_csv('datasets/friesgo2018.txt', delimiter = '|')

## Analizando el dataset (raw)

In [10]:
# Determino filas y columnas
friesgo_2018.shape

(29224, 287)

In [12]:
# Obtengo las 5 primeras filas del dataset
friesgo_2018.head()

,id,cod_provincia,region,tamanio_aglomerado,aglomerado,localidades_150,submuestra,bhcv01,bhcv02,bhcv03,...,glucemia_elevada,prevalencia_glucemia_elevada_combinada,findrisc,bimq06,bimq06_01,colesterol_elevado,prevalencia_colesterol_combinada,wf1p,wf2p,wf3p
0,1128639,2,1,1,1,1,1,3,2,1,...,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2378,4464,0
1,1709939,2,1,1,1,1,0,1,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3588,0,0
2,6874130,2,1,1,1,1,0,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2740,0,0
3,10319375,2,1,1,1,1,1,3,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,493,0,0
4,11140857,2,1,1,1,1,0,3,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,0,0


##  Evaluacion de columnas del dataset

Vamos a agrupar las columnas por temática, manteniendo ciertas columnas que permiten la identificacion del registro (como el id) y aquellas necesarias para realizar analisis, tales como variables demograficas.
Por cada temática:

*   Visualizo las primeras filas
*   Renombro columnas y evaluo los nulos
*   Calculo de porcentaje de faltantes por columna

### Busqueda y localizacion de bloques tematicos

In [ ]:
#Busco el indice de una columna para dividir los bloques tematicos
friesgo_2018.columns.get_loc('cobertura_salud')

In [ ]:
#caracteristicas del encuestado, sexo, nivel de instruccion rango etario, etc.
caracteristicas_encuestado = friesgo_2018.iloc[:,51:65]
#caracteristicas_encuestado.head(20)
calcular_nulos_y_porcentaje(caracteristicas_encuestado)

In [ ]:
saludgeneral_actfisica  = friesgo_2018.iloc[:,73:97]
#saludgeneral_actfisica.head(20)
calcular_nulos_y_porcentaje(saludgeneral_actfisica)

In [ ]:
hipertension = friesgo_2018.iloc[:,140:159]
#hipertension.head(20)
calcular_nulos_y_porcentaje(hipertension)

In [ ]:
pesocorporal = friesgo_2018.iloc[:,160:168]
#pesocorporal.head(20)
calcular_nulos_y_porcentaje(pesocorporal)

In [ ]:
alimentacion = friesgo_2018.iloc[:,169:186]
#alimentacion.head(20)
calcular_nulos_y_porcentaje(alimentacion)

In [ ]:
diabetes = friesgo_2018.iloc[:,218:236]
#diabetes.head(20)
calcular_nulos_y_porcentaje(diabetes)

In [ ]:
mediciones_antropometricas = friesgo_2018.iloc[:,252:273]
#mediciones_antropometricas.head(20)
calcular_nulos_y_porcentaje(mediciones_antropometricas)

In [ ]:
mediciones_bioquimicas = friesgo_2018.iloc[:,274:283]
#mediciones_bioquimicas.head(20)
calcular_nulos_y_porcentaje(mediciones_bioquimicas)

### Tabaquismo
Agregar detalle de columnas implicadas en este dataset.

#### Remuevo las siguientes columnas por ser calculadas en base a las respuestas de otras preguntas

* consumo_tabaco_100
* ta_paquete_y_armado
* ta_dejar_fumar
* ta_otros_productos
* hta_nofumadores
* ta_perc_publicidad
* ta_percepcion_riesgo

In [22]:
# Creo el DataFrame para tabaquismo
# Selecciono la columna 0 - correspondiente a la columna id - y el rango de preguntas para tabaquismo
tabaquismo = friesgo_2018.iloc[:, [0] + list(range(98, 132))]

In [ ]:
# Visualizo las primeras 20 filas
tabaquismo.head(20)

In [24]:
# Renombro columnas
tabaquismo.rename(columns = {
    'bita01':       'es_fumador',
    'bita02':       'edad_fuma_primera_vez',
    'bita02_99':    'edad_fuma_primera_vez_ns_nc',
    'bita03':       'fumado_100_cigarrillos',
    'bita04':       'periodicidad_fumador',
    'bita04_01':    'es_cigarrillo_armado',
    'bita04_02':    'es_cigarrillo_paquete',
    'bita05':       'marca_cigarrillos',
    'bita06_a':     'tipo_paquete_comprado',
    'bita06_b':     'cant_cigarrillos_comprados',
    'bita06_b_99':  'cant_cigarrillos_comprados_ns_nc',
    'bita06_c':     'cigarrillos_sabor',
    'bita06_d':     'cigarrillos_tipo_capsula',
    'bita07':       'dinero_pagado_ultima_compra_cigarrillos',
    'bita07_99':    'dinero_pagado_ultima_compra_cigarrillos_ns_nc',
    'bita08':       'intento_dejar_fumar',
    'bita09_01':    'es_fumador_cigarro_habano',
    'bita09_02':    'es_fumador_cigarritos',
    'bita09_03':    'es_fumador_pipa_comun',
    'bita09_04':    'es_fumador_pipa_agua',
    'bita09_05':    'es_fumador_tabaco_masticar',
    'bita09_06':    'es_fumador_cigarrillo_electronico',
    'bita10_01':    'hubo_fumadores_casa',
    'bita10_02':    'hubo_fumadores_trabajo',
    'bita10_03':    'hubo_fumadores_escuela',
    'bita10_04':    'hubo_fumadores_restaurant',
    'bita10_05':    'hubo_fumadores_hospital',
    'bita10_06':    'hubo_fumadores_otros_lugares',
    'bita11':       'publicidad_comercios',
    'bita12':       'publicidad_email',
    'bita13':       'suscripcion_cigarrera',
    'bita14':       'vio_propaganda_riesgo_fumar',
    'bita15':       'propaganda_piensa_dejar_fumar',
    'bita16':       'acuerdo_impuesto_tabaco'
}, inplace = True)

C:\Users\Biank\AppData\Local\Temp\ipykernel_4224\2009123552.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabaquismo.rename(columns = {


In [ ]:
# Visualizo nulos y porcentaje
calcular_nulos_y_porcentaje(tabaquismo)

### Ubicacion

Esta seccion contiene las siguientes columnas:
+ id
+ cod_provincia
+ region
+ tamanio_aglomerado
+ aglomerado : Aglomerado urbano de 500.000 y más habitantes
+ localidades_150 : Tamaño de localidad - 150.000 habitantes o más. (1 True 0 False)
+ submuestra : Identificación de la submuestra de los pasos 2 y 3 (*). Correspondientes a las mediciones antropometricas y bioquimicas. (De no usar las mediciones recomiendo borrar esta columna). (1 True 0 False)
(*) Ver readme

In [ ]:
# Creo una tabla aparte para ubicacion
ubicacion = friesgo_2018.iloc[:,0:7]
#ubicacion.head()

### Consumo alcohol

Este bloque tematico esta compuesto por las siguientes columnas:
+ id 
+ ha_consumido_alc COD
+ fecha_ult_cons COD
+ freq_30_ds (Int 1:7)
+ freq_30_dm (Int 0:30)
+ freq_30_nsnc (solo 99)
+ tragos_cerveza 
+ tragos_cerveza_notoma *
+ tragos_vino
+ tragos_vino_notoma *
+ tragos_bebida_fuerte
+ tragos_bebida_fuerte_notoma *
+ toma_otras_bebidas **
+ sobrecon_cant_cerveza 
+ sobrecon_cerv_notoma *
+ sobrecon_cant_vino
+ sobrecon_vino_notoma *
+ sobrecon_cant_bebfuerte 
+ sobrecon_bebfuerte_notoma *
+ sobrecon_otrasb **
+ manejo_alcholizado30,
+ aprueba_aumento_imp_alchol
+ consumo_regular_riesgo **
+ consumo_episodico_excesivo **
(*) Solo un valor 98:'No toma'
(**) 1:'Si' , 2: 'No'

In [5]:
# Creo mi tabla de consumo de alcohol
consumo_alcohol = friesgo_2018.iloc[:, [0] + list(range(195,217))]
consumo_alcohol.head()

,id,bica01,bica02,bica03_01,bica03_02,bica03_99,bica04_01_b,bica04_01_c,bica04_02_b,bica04_02_c,...,bica05_01_b,bica05_01_c,bica05_02_b,bica05_02_c,bica05_03_b,bica05_03_c,bica05_04,bica06,bica07,consumo_regular_riesgo
0,1128639,1,1.0,1.0,NaN,NaN,1.0,NaN,NaN,98.0,...,3.0,NaN,1.0,NaN,NaN,98.0,2.0,2.0,2,2.0
1,1709939,1,1.0,NaN,1.0,NaN,NaN,98.0,1.0,NaN,...,NaN,98.0,NaN,98.0,NaN,98.0,2.0,2.0,2,2.0
2,6874130,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0
3,10319375,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
4,11140857,1,1.0,1.0,NaN,NaN,1.5,NaN,NaN,98.0,...,NaN,98.0,NaN,98.0,NaN,98.0,2.0,2.0,99,2.0


In [4]:
consumo_alcohol.columns

Index(['id', 'bica01', 'bica02', 'bica03_01', 'bica03_02', 'bica03_99',
       'bica04_01_b', 'bica04_01_c', 'bica04_02_b', 'bica04_02_c',
       'bica04_03_b', 'bica04_03_c', 'bica04_04', 'bica05_01_b', 'bica05_01_c',
       'bica05_02_b', 'bica05_02_c', 'bica05_03_b', 'bica05_03_c', 'bica05_04',
       'bica06', 'bica07', 'consumo_regular_riesgo'],
      dtype='object')

In [6]:
# Renombro las columnas
consumo_alcohol.rename( columns = {
    'bica01': 'ha_consumido_alc',
    'bica02': 'fecha_ult_cons',
    'bica03_01': 'freq_30_ds',
    'bica03_02': 'freq_30_dm',
    'bica03_99': 'freq_30_nsnc',
    'bica04_01_b': 'tragos_cerveza',
    'bica04_01_c': 'tragos_cerveza_notoma',
    'bica04_02_b': 'tragos_vino',
    'bica04_02_c': 'tragos_vino_notoma',
    'bica04_03_b': 'tragos_bebida_fuerte',
    'bica04_03_c': 'tragos_bebida_fuerte_notoma',
    'bica04_04': 'toma_otras_bebidas',
    'bica05_01_b': 'sobrecon_cant_cerveza',
    'bica05_01_c': 'sobrecon_cerv_notoma',
    'bica05_02_b': 'sobrecon_cant_vino',
    'bica05_02_c': 'sobrecon_vino_notoma',
    'bica05_03_b': 'sobrecon_cant_bebfuerte',
    'bica05_03_c': 'sobrecon_bebfuerte_notoma',
    'bica05_04': 'sobrecon_otrasb',
    'bica06': 'manejo_alcholizado30',
    'bica07': 'aprueba_aumento_imp_alchol'
}, inplace= True)

C:\Users\Biank\AppData\Local\Temp\ipykernel_4224\2100149357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  consumo_alcohol.rename( columns = {


### Colesterol

Esta seccion contiene las siguientes columnas :
+ medido_colesterol **
+ fecha_med_colesterol COD
+ colesterol_alto **
+ tratamiento_medico_col *
+ trat_hab_col *
+ trat_farmac_col *
+ control_colesterol **
+ prevalencia_colesterol **

(*)No voy a hacer un diccionario para los siguientes ya que se repite el patron= 1: No, 2:Si 
(**) Anterior + codigo 99:Ns/Nc
(COD) Tiene codificacion

In [6]:
# Creo mi tabla de tematica: colesterol
colesterol = friesgo_2018.iloc[:, [0] + list(range(187,195))]

Index(['id', 'bico01', 'bico02', 'bico03', 'bico04', 'bico05_01', 'bico05_02',
       'control_colesterol', 'prevalencia_colesterol'],
      dtype='object')

In [ ]:
# Renombro las columnas 
colesterol.rename( columns={
    'bico01': 'medido_colesterol',
    'bico02': 'fecha_med_colesterol',
    'bico03': 'colesterol_alto',
    'bico04': 'tratamiento_medico_col',
    'bico05_01': 'trat_habitos_col',
    'bico05_02': 'trat_farmac_col',
    'control_colesterol': 'autocontrol_col'
}, inplace= True)

In [9]:
#colesterol.columns
#colesterol.head()
#calcular_nulos_y_porcentaje(colesterol)

,id,medido_colesterol,fecha_med_colesterol,colesterol_alto,tratamiento_medico_col,trat_habitos_col,trat_farmac_col,autocontrol_col,prevalencia_colesterol
0,1128639,1,2.0,2,NaN,NaN,NaN,NaN,2.0
1,1709939,1,2.0,1,2.0,NaN,NaN,1.0,1.0
2,6874130,1,2.0,2,NaN,NaN,NaN,NaN,2.0
3,10319375,1,2.0,1,1.0,2.0,1.0,1.0,1.0
4,11140857,1,1.0,2,NaN,NaN,NaN,NaN,2.0


### Salud general y actividad fisica

Según el diccionario de registros correspondiente al dataset, las variables (nombres de columnas) que corresponden a los temas indicados son: 

*SALUD GENERAL*
+ "bisg01", 
+ "bisg02",
+ "bisg03",
+ "bisg04",
+ "bisg05",
+ "bisg06",

*ACTIVIDAD FÍSICA*
+ "biaf01",
+ "biaf02_m",
+ "biaf02_99",
+ "biaf03",
+ "biaf04_m",
+ "biaf04_99",
+ "biaf05",
+ "biaf06_m",
+ "biaf06_99",
+ "biaf07_m",
+ "biaf07_99",
+ "biaf08",
+ "biaf09",
+ "biaf10_01",
+ "biaf10_02",
+ "biaf10_03",
+ "biaf10_04",
+ "nivel_actividad_fisica",
+ "barreras_actividad_fisica"

In [ ]:
#Formando el subset salud_y_af y renombrando sus variables
salud_y_af = friesgo_2018.iloc[:, [0] + list(range(73,98))]
#salud_y_af.head()

In [41]:

# Renombro columnas
salud_y_af.rename(columns = {
    #SALUD GENERAL 
"bisg01" : "salud_actual",
"bisg02" : "nivel_de_movilidad",
"bisg03" : "capacidad_cuidado_personal",
"bisg04" : "capacidad_actividades_cotidianas",
"bisg05" : "dolor_malestar_general",
"bisg06" : "ansiedad_depresion",

#ACTIVIDAD FÍSICA  af
"biaf01" : "af_intensa_veces_por_semana",
"iaf02_m" : "af_intensa_min_totales_por_semana",
"biaf02_99" : "af_intensa_min_totales_por_semana_nsnc",
"biaf03" : "af_moderada_veces_por_semana",
"biaf04_m" : "af_moderada_min_totales_por_semana",
"biaf04_99" : "actividad_fisica_moderada_min_totales_por_semana_nsnc",
"biaf05" : "caminata_veces_por_semana",
"biaf06_m" : "caminata_min_totales_por_semana",
"biaf06_99" : "caminata_min_totales_por_semana_nsnc",
"biaf07_m" : "tiempo_sentado_min_por_dia",
"biaf07_99" : "tiempo_sentado_min_por_dia_nsnc",
"biaf08" : "razon_ninguna_actividad_la_ultima_semana",
"biaf09" : "razon_poca_actividad_la_ultima_semana",
"biaf10_01" : "af_fue_actividad_domestica",
"biaf10_02" : "af_fue_actividad_laboral",
"biaf10_03" : "af_fue_traslado",
"biaf10_04" : "af_fue_deporte",
"nivel_actividad_fisica" : "nivel_af",
"barreras_actividad_fisica" : "barreras_af"

}, inplace = True)

C:\Users\Biank\AppData\Local\Temp\ipykernel_4224\3558483967.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salud_y_af.rename(columns = {


In [42]:
#salud_y_af.columns

Index(['id', 'salud_actual', 'nivel_de_movilidad',
       'capacidad_cuidado_personal', 'capacidad_actividades_cotidianas',
       'dolor_malestar_general', 'ansiedad_depresion',
       'af_intensa_veces_por_semana', 'biaf02_m',
       'af_intensa_min_totales_por_semana_nsnc',
       'af_moderada_veces_por_semana', 'af_moderada_min_totales_por_semana',
       'actividad_fisica_moderada_min_totales_por_semana_nsnc',
       'caminata_veces_por_semana', 'caminata_min_totales_por_semana',
       'caminata_min_totales_por_semana_nsnc', 'tiempo_sentado_min_por_dia',
       'tiempo_sentado_min_por_dia_nsnc',
       'razon_ninguna_actividad_la_ultima_semana',
       'razon_poca_actividad_la_ultima_semana', 'af_fue_actividad_domestica',
       'af_fue_actividad_laboral', 'af_fue_traslado', 'af_fue_deporte',
       'nivel_af', 'barreras_af'],
      dtype='object')

## Analisis / trabajo sobre base de datos "UTIL"

En esta seccion:
+ A partir de las tablas limpias crearemos el dataset a utilizar
+ Realizaremos 